In [5]:
from selenium import webdriver
import time
from webdriver_manager import driver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime as dt
import datetime




options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver =webdriver.Chrome(ChromeDriverManager().install(),options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})
driver.execute_cdp_cmd("Network.enable", {})
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})

main=[]



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\Kavitha\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [6]:

dates=['22-10-2021','23-10-2021','24-10-2021','25-10-2021','26-10-2021','27-10-2021','28-10-2021','29-10-2021','30-10-2021','31-10-2021','01-11-2021','02-11-2021','03-11-2021']
l=[]
i=0
while len(l)<1500 and i< len(dates):
    date1=dates[i]
    #21%2F10%2F2021
    date='%2F'.join(date1.split('-'))
    url=f'https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=DEL&originCountry=IN&destination=BOM&destinationCountry=IN&flight_depart_date={date}&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=682925483509'
    driver.get(url)
    time.sleep(40)
    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    for e in driver.find_elements_by_xpath('//div[@class="flightItem border-shadow pr"]'):
        try:
            airline_name=e.find_element_by_xpath('.//span[@class="i-b text ellipsis"]').text
            departureTime=e.find_element_by_xpath('.//div[@autom="departureTimeLabel"]').text
            arrivalTime=e.find_element_by_xpath('.//p[@autom="arrivalTimeLabel"]').text
            departure_city=e.find_element_by_xpath('.//p[@class="fs-10 font-lightgrey no-wrap city ellipsis"]').text
            arrival_city=e.find_elements_by_xpath('.//p[@class="fs-10 font-lightgrey no-wrap city ellipsis"]')[1].text
            duration=e.find_element_by_xpath('.//p[@autom="durationLabel"]').text
            try:
                stops=e.find_element_by_xpath('.//span[@class="dotted-borderbtm"]').text
            except Exception as err:
                #print(err)
                stops='Non-stop'
                
            if '+ 1' in arrivalTime:
                arrivalTime=arrivalTime.replace(r'+ 1 day','').replace(r'\n','').strip()
                date_dt2 = dt.strptime(date1, '%d-%m-%Y')
                date_dt2 += datetime.timedelta(days=1)
                arrivalTime += (' '+str(date_dt2).replace(' 00:00:00',''))

            price=e.find_element_by_xpath('.//div[@class="i-b tipsy fare-summary-tooltip fs-18"]').text
            d={
                'Airline':airline_name,
                'Date_of_Journey':date1,
                'Source':departure_city,
                'Destination': arrival_city,
                'Dep_Time': departureTime,
                'Arrival_Time':arrivalTime.replace(r'\n+ 1 day','').replace(r'\n+ 2 day',''),
                'Duration':duration,
                'Total_Stops' :stops,
                'Price':price
                }
            #pprint.pprint(d)
            l.append(d)
        except:
            pass
    i+=1
    
driver.quit()
df=pd.DataFrame(l)
df.to_excel('output_flight_data.xlsx',index=False)